### Create anndata object for Ishikawa, 2022 healthy ascending colon that contains LGR5+ stem cells

- **Developed by:** Anna Maguza
- **Affilation:** Faculty of Medicine, Würzburg University
- **Date of creation:** 4th October 2024
- **Last modified date:** 4th October 2024

In [31]:
import pandas as pd
import anndata as ad
import scanpy as sc
import numpy as np
from scipy.io import mmread
from scipy.sparse import coo_matrix
import scipy as sci

In [ ]:
def X_is_raw(adata):
    return np.array_equal(adata.X.sum(axis=0).astype(int), adata.X.sum(axis=0))

In [32]:
barcodes = pd.read_csv('data/Ishikawa_2022/barcodes.tsv', sep='\t', header=None)
features = pd.read_csv('data/Ishikawa_2022/features.tsv', sep='\t', header=None)

annotation = pd.read_csv('data/Ishikawa_2022/Ishikawa_celltype_annotation.csv', header=0)
embedding = pd.read_csv('data/Ishikawa_2022/Ishikawa_umap_embedding.csv', header=0)

In [33]:
matrix = mmread('data/Ishikawa_2022/matrix.mtx')
matrix = sci.sparse.csr_matrix(matrix)
matrix = np.transpose(matrix)

In [34]:
barcodes.rename(columns={0: 'cell_id'}, inplace=True)
features.rename(columns={0: 'gene_id', 1: 'gene_name', 2: 'gene_type'}, inplace=True)

In [35]:
adata = ad.AnnData(X=matrix, obs=barcodes, var=features)

/home/amaguza/miniforge3/envs/scanpy_env/lib/python3.11/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/home/amaguza/miniforge3/envs/scanpy_env/lib/python3.11/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [49]:
X_is_raw(adata)

True

In [36]:
embedding.rename(columns={'Unnamed: 0': 'cell_id'}, inplace=True)
annotation.rename(columns={'Unnamed: 0': 'cell_id'}, inplace=True)

In [37]:
adata = adata[adata.obs['cell_id'].isin(embedding['cell_id'])]

/home/amaguza/miniforge3/envs/scanpy_env/lib/python3.11/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


In [38]:
adata.obsm['X_umap'] = embedding.iloc[:, 1:3].values

In [39]:
adata.obs = pd.merge(adata.obs, annotation, on='cell_id')

In [ ]:
sc.set_figure_params(figsize=(7, 7), dpi=180)
sc.pl.umap(adata, color='celltype')


In [48]:
adata.write('data/Ishikawa_2022/Ishikawa_2022_raw_filtered.h5ad')